In [328]:
import tensorflow as tf
import numpy as np

import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

%matplotlib inline
from sutils import *
import os, json
from glob import glob

from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
#from keras.applications.inception_v4 import InceptionV4
from keras.preprocessing import image
from keras.models import Model,load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input


In [76]:
print(keras.__version__)
print(tf.__version__)

NameError: name 'keras' is not defined

In [329]:
path = "data/face/Emotion/Toons/"
batch_size=4
no_of_epochs=10

In [330]:
# Prepare images for training in batches
# NB: They must be in subdirectories named based on their category
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2)

Found 42847 images belonging to 7 classes.
Found 12920 images belonging to 7 classes.


In [331]:
# create the base pre-trained model
#base_model = InceptionV3(weights='imagenet', include_top=False)
#base_model = VGG16(weights='imagenet', include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False)

94461952/94653016 [============================>.] - ETA: 0s

In [332]:
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, None, None, 3) 0           input_7[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, None, None, 64 256         conv1[0][0]                      
___________________________________________________________________________________________

In [333]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(7, activation='softmax')(x)

In [334]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [335]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    print(layer.name)
    layer.trainable = False

input_7
zero_padding2d_1
conv1
bn_conv1
activation_377
max_pooling2d_17
res2a_branch2a
bn2a_branch2a
activation_378
res2a_branch2b
bn2a_branch2b
activation_379
res2a_branch2c
res2a_branch1
bn2a_branch2c
bn2a_branch1
add_1
activation_380
res2b_branch2a
bn2b_branch2a
activation_381
res2b_branch2b
bn2b_branch2b
activation_382
res2b_branch2c
bn2b_branch2c
add_2
activation_383
res2c_branch2a
bn2c_branch2a
activation_384
res2c_branch2b
bn2c_branch2b
activation_385
res2c_branch2c
bn2c_branch2c
add_3
activation_386
res3a_branch2a
bn3a_branch2a
activation_387
res3a_branch2b
bn3a_branch2b
activation_388
res3a_branch2c
res3a_branch1
bn3a_branch2c
bn3a_branch1
add_4
activation_389
res3b_branch2a
bn3b_branch2a
activation_390
res3b_branch2b
bn3b_branch2b
activation_391
res3b_branch2c
bn3b_branch2c
add_5
activation_392
res3c_branch2a
bn3c_branch2a
activation_393
res3c_branch2b
bn3c_branch2b
activation_394
res3c_branch2c
bn3c_branch2c
add_6
activation_395
res3d_branch2a
bn3d_branch2a
activation_396
re

In [336]:
for layer in model.layers:
    print(layer.name)
    print(layer.trainable)

input_7
False
zero_padding2d_1
False
conv1
False
bn_conv1
False
activation_377
False
max_pooling2d_17
False
res2a_branch2a
False
bn2a_branch2a
False
activation_378
False
res2a_branch2b
False
bn2a_branch2b
False
activation_379
False
res2a_branch2c
False
res2a_branch1
False
bn2a_branch2c
False
bn2a_branch1
False
add_1
False
activation_380
False
res2b_branch2a
False
bn2b_branch2a
False
activation_381
False
res2b_branch2b
False
bn2b_branch2b
False
activation_382
False
res2b_branch2c
False
bn2b_branch2c
False
add_2
False
activation_383
False
res2c_branch2a
False
bn2c_branch2a
False
activation_384
False
res2c_branch2b
False
bn2c_branch2b
False
activation_385
False
res2c_branch2c
False
bn2c_branch2c
False
add_3
False
activation_386
False
res3a_branch2a
False
bn3a_branch2a
False
activation_387
False
res3a_branch2b
False
bn3a_branch2b
False
activation_388
False
res3a_branch2c
False
res3a_branch1
False
bn3a_branch2c
False
bn3a_branch1
False
add_4
False
activation_389
False
res3b_branch2a
False
b

In [337]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [338]:
test = batches.next()

len(test[1][0])
test[1][2]

array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.], dtype=float32)

In [362]:
hist = model.fit_generator(batches, steps_per_epoch=8, epochs=no_of_epochs,verbose=1,
                validation_data=val_batches, validation_steps=3)

Epoch 1/10
8/8 [==============================] - 46s - loss: 0.2442 - acc: 0.9062 - val_loss: 0.4638 - val_acc: 0.8333
Epoch 2/10
8/8 [==============================] - 45s - loss: 0.2829 - acc: 0.9062 - val_loss: 0.9879 - val_acc: 0.6250
Epoch 3/10
8/8 [==============================] - 47s - loss: 0.3539 - acc: 0.8438 - val_loss: 0.7864 - val_acc: 0.7917
Epoch 4/10
8/8 [==============================] - 45s - loss: 0.6819 - acc: 0.7188 - val_loss: 0.9759 - val_acc: 0.6667
Epoch 5/10
8/8 [==============================] - 45s - loss: 0.2970 - acc: 0.9688 - val_loss: 1.5683 - val_acc: 0.4583
Epoch 6/10
8/8 [==============================] - 45s - loss: 0.4472 - acc: 0.7500 - val_loss: 0.8882 - val_acc: 0.6250
Epoch 7/10
8/8 [==============================] - 46s - loss: 0.4534 - acc: 0.8125 - val_loss: 0.5154 - val_acc: 0.9167
Epoch 8/10
8/8 [==============================] - 45s - loss: 0.1886 - acc: 0.9375 - val_loss: 0.8610 - val_acc: 0.7500
Epoch 9/10
8/8 [========================

In [363]:
metrics = model.evaluate_generator(val_batches,10,10,workers=1,pickle_safe=False)
print("model accuracy:",metrics[1])

model accuracy: 0.525


In [348]:
model.save('face-try.hdf5')

In [349]:
model = load_model('face-try.hdf5')

In [350]:
image_path = '/Users/Neeraj/Downloads/Test_'

In [395]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import numpy as np

img_path = os.path.join(image_path, 'ray_sadness_250.png')
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print('Input image shape:', x.shape)

Input image shape: (1, 224, 224, 3)


In [396]:
preds = model.predict(x)
print('Predicted:', preds)

Predicted: [[  3.12552652e-06   3.48199553e-15   9.99959230e-01   7.49403930e-15
    5.69218317e-17   3.77234683e-05   5.76809583e-14]]


In [397]:
j=0;
i,j=np.unravel_index(preds.argmax(), preds.shape)
print("Max",preds[i,j],j)

Max 0.999959 2


In [398]:
if j==0:
    print("Anger")
elif j ==1:
    print("Disgust")
elif j==2:
    print("Fear")
elif j==3:
    print("Joy")
elif j==4:
    print("Neutral")
elif j==5:
    print("Sadness")
else:
    print("Surprise")


Fear


In [399]:
img.show()